In [1]:
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize,PorterStemmer
import re
import string



In [2]:
df = pd.read_csv('reviews_fin.csv')


In [3]:
df.head()

,name,rating,text
0,0031 Boutique Hotel Restaurant,5.0,At 0031 you come to the right place. Everythin...
1,0031 Boutique Hotel Restaurant,5.0,Just came back from my trip to Cumbuco. It was...
2,0031 Boutique Hotel Restaurant,5.0,I was 6 weeks in Cumbuco. 0031 is the best pla...
3,0031 Boutique Hotel Restaurant,5.0,everything was very nice. we had a lovely stay...
4,0031 Boutique Hotel Restaurant,5.0,For the third time in a row we've been to this...


In [ ]:
# filter hotels names
names = df['name']
names.drop_duplicates(inplace=True)
# flatten reviews
hotels_reviews = dict()
hotels_reviews = {k:"" for k in names}
hotels_ratings = {k:0 for k in names}

In [ ]:
for _,row in df.iterrows():
    hotels_reviews[row['name']] = hotels_reviews[row['name']] +row['text']
    hotels_ratings[row['name']] = hotels_ratings[row['name']] + row['rating']

In [ ]:
ratings_df= pd.DataFrame(columns=['name','rating'],data=hotels_ratings.items())
reviews_df = pd.DataFrame(columns=['name','review'],data=hotels_reviews.items())
hotels_df = pd.merge(ratings_df,reviews_df,on='name')
hotels_df.plot.hist(bins=10,alpha=1)

hotels_df.shape

In [ ]:
hotels_df = pd.read_csv('hotels_reviews.csv')

In [ ]:

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
def preprocess(text):
    # Lower case
    text = text.lower()
    # remove punct    
    table = str.maketrans('', '', string.punctuation)
    text = text.translate(table)
    text = text.split()
    
    # remove digits and strange symbol 
    text = [re.sub("(\d+|€+)",'',word) for word in text]
    
    text = " ".join(text)
   
    # remove strange encoding chars
    filters = " ".join(filter(lambda char: char not in string.printable, text))
    if len(filters) > 0:
        text = re.sub(f'[{filters}*]',' ',text)
    
    # tokinaztion
    text = word_tokenize(text)
  
    # remove stopwords and single chars
    text = [t for t in text if t not in stop_words and len(t) > 1]
#     lemmatizer = wordnet.WordNetLemmatizer()
#     text = [lemmatizer.lemmatize(w) for w in text]
#     hashtags.extend([word for word in text if "#" in word])
#     stemming
    text = [stemmer.stem(w) for w in text]
    
    return " ".join(text)

In [ ]:
hotels_df['review_cleaned'] = hotels_df['review'].apply(preprocess)

In [ ]:
hotels_df.to_csv('hotels_reviews.csv',index=False)